# Multiple Linear Regression

This week we will be get our hands on some real data and create a multiple linear regression model from the data. First you need to create a new directory called ```data``` which we will use to store the data. In the command line run:

```mkdir data```

Download the pickle file ```winequality-white.pickle``` and save it in this new data folder.

We will use scikit-learn to do the linear regression for us, then there will be a little exercise for you to use numpy to compute the least squares estimate and compare wether your code gives the same set of weights as scikit-learn.

For more details about scikit-learn see : (https://scikit-learn.org/stable/)

In [1]:
# all the imports we need
import matplotlib.pyplot as plt
import _pickle as cp
import numpy as np
from numpy.linalg import inv
from sklearn import linear_model

### Loading in the data

First we load in the data (feature) matrix ```X``` and the data (output) vector ```y```.

In [2]:
# load the full dataset
# data matrix X and data vector y
X, y = cp.load(open('data/winequality-white.pickle', 'rb'))
# print the shape of the dataset
print(f'data matrix shape {X.shape}', f'data vector shape {y.shape}')

data matrix shape (4898, 11) data vector shape (4898,)


### Data preprocessing

First we need to do some simple data preprocessing. Remember we need to add a column of all ones to model the bias vector giving us a data matrix with shape N, D+1

In [3]:
# N refers to the number of datapoints, D refers to the dimension of each datapoint (how many features it has)
N, D = X.shape

print(f'data matrix has {N} data points, with {D} features')
X = np.concatenate([np.ones(N).reshape(-1,1), X], axis=1)

N, D = X.shape
print(f'data matrix has {N} data points, with {D} features')

# The first column of X is now entire ones, uncomment the following print statement to see.
#print(X[:,0])

data matrix has 4898 data points, with 11 features
data matrix has 4898 data points, with 12 features


### Splitting the data up

It is standard practice to split up your dataset into training data and test data. The training data is used to train your model, or in this instance calculate the least squares estimate, the test data is then kept aside to evaluate your model on unseen data. This is very important! As if there is a big discrepency and accuracy between the training and test data then this indicated you have overfit to your training data. If this is the case then you might need to consider a different type of model or use additional *regularization* techniques to prevent overfitting.

In [4]:
# N refers to the number of datapoints, D refers to the dimension of each datapoint (how many features it has)
N, D = X.shape

# split the data set into 80% train and 20% test data
N_train = int(0.8 * N)
N_test = N - N_train

X_train = X[:N_train]
y_train = y[:N_train]
X_test = X[N_train:]
y_test = y[N_train:]

### Using scikit-learn

We can use scikit-learn to do multiple linear regression for us under the hood.

In [5]:
reg = linear_model.LinearRegression()

# fit a linear model to the training data
reg.fit(X_train, y_train)

# scikit learn stores the feature weights and bias (y-intercept) in different attributes
# we just need to reconcile them into one vector
weights = np.array(reg.coef_)
weights[0] = np.array(reg.intercept_)

# copying the weights for later
scikit_learn_weights = np.copy(weights)

# compute the y hat estimates for the train data 
y_hat_train = np.matmul(X_train, weights)
# compute the y hat estimates for the test data
y_hat_test = np.matmul(X_test, weights)

# compute the training dataset error
train_mse = ((y_train - y_hat_train)**2).mean()
# compute the test dataset error
test_mse = ((y_test - y_hat_test)**2).mean()

# print out the train and test error to see if the model has overfit or not
print("Train Error (MSE): {:.4f}, Test Error (MSE): {:.4f}".format(train_mse, test_mse))

Train Error (MSE): 0.5640, Test Error (MSE): 0.5607


In [6]:
# print out the weights of the model
print(weights)

[ 1.46625479e+02  6.79734355e-02 -1.84189093e+00 -3.25205250e-02
  8.07889873e-02 -3.77326708e-01  3.46732924e-03 -8.41204098e-05
 -1.46636726e+02  6.58475903e-01  5.78204430e-01  1.97896436e-01]


### Now its your turn [Exercise]

Now it is up to you to derive the least squares estimate for the training data ```X_train``` and ```y_train``` using only numpy commands.

If you get it right then you should get the same values as the weights output by scikit-learn

- To invert a matrix use ```inv(matrix)```
- To multiple two matrices together use ```np.matmul(matrix_1, matrix_2)```
- To transpose a matrix use ```np.transpose(matrix)``` or ```matrix.transpose()```

In [7]:
# your code goes here

X_T = np.transpose(X_train)

X_T_X = np.matmul(X_T, X_train)

X_T_X_inv = inv(X_T_X)

weights = np.matmul(np.matmul(X_T_X_inv, X_T), y_train)

In [8]:
# print out the weights of the model 
print(weights)

[ 1.46625479e+02  6.79734355e-02 -1.84189093e+00 -3.25205252e-02
  8.07889873e-02 -3.77326708e-01  3.46732924e-03 -8.41204095e-05
 -1.46636726e+02  6.58475903e-01  5.78204430e-01  1.97896436e-01]


In [9]:
if np.sum(weights - scikit_learn_weights) < 1e-10:
    print("Well done the weights you computed are correct!!!")
else:
    print("The weights you computed are not quite right try again.")

Well done the weights you computed are correct!!!


In [10]:
# compute the y hat estimates for the train data
y_hat_train = np.matmul(X_train, weights)
# compute the y hat estimates for the test data
y_hat_test = np.matmul(X_test, weights)

# compute the training dataset error
train_mse = ((y_train - y_hat_train)**2).mean()
# compute the test dataset error
test_mse = ((y_test - y_hat_test)**2).mean()

# the train error and test error should be the same as before if you've correctly computed the same weights (duh!)
print("Train Error (MSE): {:.4f}, Test Error (MSE): {:.4f}".format(train_mse, test_mse))

Train Error (MSE): 0.5640, Test Error (MSE): 0.5607


## Additional exercises [centering the data]

For regression it is very common to first center the data around 0 with standard deviation 1. This can help massively with the numerical stability of matrix inversions, understanding the relative effect of each weight and for effective hyperparameter tuning of certain regularization techniques.

If you still have time I would like you to try to first center the data around 0 and normalize the data so it has standard deviation 1. You might have seen this before, but a data point ```y``` can be centred by first subtracting the data mean and then dividing by the data standard deviation.

```y_centred = (y - y_mean) / y_std```

You need to center the data for both the (output) data vector ```y``` and each of the D features in the (feature) data matrix ```X``` 

In [11]:
# first let's reload the data in
# load the full dataset
# data matrix X and data vector y
X, y = cp.load(open('data/winequality-white.pickle', 'rb'))
# print the shape of the dataset
print(f'data matrix shape {X.shape}', f'data vector shape {y.shape}')

data matrix shape (4898, 11) data vector shape (4898,)


#### No bias / y-intercept is needed

Since the (output) data vector ```y``` will have mean 0 we no longer need a bias or y-intercept 

In [12]:
# let's split up the data again into training and test datasets
# N refers to the number of datapoints, D refers to the dimension of each datapoint (how many features it has)
N, D = X.shape

# split the data set into 80% train and 20% test data
N_train = int(0.8 * N)
N_test = N - N_train

X_train = X[:N_train]
y_train = y[:N_train]
X_test = X[N_train:]
y_test = y[N_train:]

### Now its your turn [Exercise]

Now its up to you to center the data given the means and standard deviations of the output feature and all the input features

In [13]:
# let's compute the mean and standard deviation of the training data
# you can use np.mean(matrix) or matrix.mean(), similarly np.std(matrix) or matrix.std()
# first for the output feature

# your code goes here
y_mean = np.mean(y_train)
y_std = np.std(y_train)

# now for the input features
# using axis=0 computes the means along the columns not for the whole matrix
X_mean = np.mean(X_train, axis=0)
X_std = np.std(X_train, axis=0)

In [14]:
# your code goes here
y_train_centered = (y_train - y_mean) / y_std
y_test_centered = (y_test - y_mean) / y_std

X_train_centered = (X_train - X_mean) / X_std
X_test_centered = (X_test - X_mean) / X_std

### Using scikit-learn

In [15]:
reg = linear_model.LinearRegression()

# fit a linear model to the centered training data
reg.fit(X_train_centered, y_train_centered)

# we don't need to reconcile the feature weights and bias/y-intercept anymore since the data is centered
weights = np.array(reg.coef_)

# copying the weights for later
scikit_learn_weights = np.copy(weights)

# compute the y hat estimates for the train data 
# we need to unscale our predictions back to the range of our original data
y_hat_train = np.matmul(X_train_centered, weights) * y_std + y_mean
# compute the y hat estimates for the test data
y_hat_test = np.matmul(X_test_centered, weights) * y_std + y_mean

# compute the training dataset error
train_mse = ((y_train - y_hat_train)**2).mean()
# compute the test dataset error
test_mse = ((y_test - y_hat_test)**2).mean()

# print out the train and test error to see if the model has overfit or not
print("Train Error (MSE): {:.4f}, Test Error (MSE): {:.4f}".format(train_mse, test_mse))

Train Error (MSE): 0.5640, Test Error (MSE): 0.5607


In [16]:
# print out the weights of the model 
print(weights)

[ 0.06471503 -0.20807204 -0.00458464  0.46417137 -0.00927574  0.06727517
 -0.00404645 -0.49575872  0.11203058  0.07412779  0.27514103]


### Using numpy

In [17]:
# your code goes here

X_T = np.transpose(X_train_centered)

X_T_X = np.matmul(X_T, X_train_centered)

X_T_X_inv = inv(X_T_X)

weights = np.matmul(np.matmul(X_T_X_inv, X_T), y_train_centered)

In [18]:
# print out the weights of the model 
print(weights)

[ 0.06471503 -0.20807204 -0.00458464  0.46417137 -0.00927574  0.06727517
 -0.00404645 -0.49575872  0.11203058  0.07412779  0.27514103]


In [19]:
if np.sum(weights - scikit_learn_weights) < 1e-10:
    print("Well done the weights you computed are correct!!!")
else:
    print("The weights you computed are not quite right try again.")

Well done the weights you computed are correct!!!


In [20]:
# compute the y hat estimates for the train data
y_hat_train = np.matmul(X_train_centered, weights) * y_std + y_mean
# compute the y hat estimates for the test data
y_hat_test = np.matmul(X_test_centered, weights) * y_std + y_mean

# compute the training dataset error
train_mse = ((y_train - y_hat_train)**2).mean()
# compute the test dataset error
test_mse = ((y_test - y_hat_test)**2).mean()

# the train error and test error should be the same as before if you've correctly computed the same weights (duh!)
print("Train Error (MSE): {:.4f}, Test Error (MSE): {:.4f}".format(train_mse, test_mse))

Train Error (MSE): 0.5640, Test Error (MSE): 0.5607
